In [1]:
import json
from neo4j import GraphDatabase
import numpy as np
import pandas as pd
from pprint import pprint
import time
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

## Senzing resolved data

Prior to running this notebook, you need to have run the entity resolution within the Senzing Docker container.  Here I have export the results of that run to the file `entities.json`.  You can see below what that file looks like.  We will use this then to create relationships between resolved and related entities.

In [2]:
data = []

with open('./entities.json') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
pprint(data[0])

## Resolved entities

For ease, I will create a dataframe just of the resolved entities and then one for the related entities.  Note that there can be multiple entities assigned to each resolved entity (see the example 2 lines down).  

Note that for ease in visualization (among other things), I am going to create a new node type called `:Entity`, which will contain the resolved entities.  This will serve as a sort of "anchor node" for the clusters we will see form in graph.

In [4]:
resolved_ls = []

for el in data:
    if len(el['RESOLVED_ENTITY']['RECORDS']) > 1:
        resolved_ls.append(el)

In [ ]:
pprint(resolved_ls[3])

In [ ]:
tup_ls = []
for el in resolved_ls: 
    source = el['RESOLVED_ENTITY']['RECORDS'][0]
    source_uid = str(el['RESOLVED_ENTITY']['RECORDS'][0]['DATA_SOURCE']) + '.' + str(el['RESOLVED_ENTITY']['RECORDS'][0]['RECORD_ID'])
    for target in el['RESOLVED_ENTITY']['RECORDS'][1:]:
        #tar = target
        target_uid = str(target['DATA_SOURCE']) + '.' + str(target['RECORD_ID'])
        tup = (el['RESOLVED_ENTITY']['RECORDS'][0]['DATA_SOURCE'],
               source['ENTITY_DESC'], 
               source_uid, 
               target['DATA_SOURCE'],
               target['ENTITY_DESC'], 
               target_uid)
        tup_ls.append(tup)

resolved_rel_df = pd.DataFrame(tup_ls, columns=['source_data_source', 'source_desc', 'source_uid',
                                                'target_data_source', 'target_desc', 'target_uid'])
resolved_rel_df.head()

## Related entities

Similar to the above we need to create the relationships between the anchor node and the related entities.  

In [ ]:
tup_ls = []

for el in data:
    #pprint(el)
    source = el['RESOLVED_ENTITY']['RECORDS'][0]
    source_data_source = source['DATA_SOURCE']
    source_desc = source['ENTITY_DESC']
    source_uid = str(source_data_source) + '.' + str(source['RECORD_ID'])
    for target in el['RELATED_ENTITIES']:
        for la in target['RECORDS']:
            target_uid = str(la['DATA_SOURCE']) + '.' + str(la['RECORD_ID'])
            tup = (source_uid,
                   source_data_source,
                   source_desc,
                   la['DATA_SOURCE'],
                   target['MATCH_KEY'],
                   target['MATCH_LEVEL'],
                   target['MATCH_LEVEL_CODE'],
                   target['IS_AMBIGUOUS'],
                   target['IS_DISCLOSED'],
                   target_uid)
            tup_ls.append(tup)

related_rel_df = pd.DataFrame(tup_ls, columns=['source_uid', 'source_data_source', 'source_desc', 
                                           'target_data_source', 'match_key', 'match_level', 'match_level_code', 'is_ambiguous',
                                           'is_disclosed', 'target_uid'])
related_rel_df.head()

In [ ]:
resolved_rel_df.shape, related_rel_df.shape

### About this bit

This is where we set up the actual `:Entity`/anchor nodes.

In [ ]:
keep_columns = ['source_data_source', 'source_desc', 'source_uid']
ent_df = resolved_rel_df[keep_columns]
ent_df_dedup = ent_df.drop_duplicates()
ent_df.shape, ent_df_dedup.shape

In [ ]:
ent_df_dedup.head()

## Establish connection to Neo4j

You could be running Neo4j in any variety of places. This demo shows running it locally. However, if you are hosting it somewhere else, replace the bolt address with yours. Also update your password below.

In [11]:
URI = "bolt://localhost:7687"
USER = "neo4j"
PWD = " " # Put your password here
AUTH = (USER, PWD)

In [12]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
        

def insert_data(query, rows, batch_size=1000):

    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = neo_conn.query(query, 
                             parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result


neo_conn = Neo4jConnection(uri=URI, user=USER, pwd=PWD)

In [ ]:
neo_conn.query('CREATE CONSTRAINT ent IF NOT EXISTS FOR (e:Entity) REQUIRE e.uid IS UNIQUE')

In [21]:
def add_ent(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               MERGE (:Entity {uid: row.source_uid,
                               name: row.source_desc,
                               data_source: row.source_data_source})
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_ent_rel(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               MATCH (source {uid: row.source_uid})
               MATCH (target {uid: row.target_uid})
               MERGE (source)-[:RESOLVES]->(target)
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_sg_related_rel(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               MATCH (source:Entity {uid: row.source_uid})
               MATCH (target:SGEntity {uid:row.target_uid})
               MERGE (target)-[:RELATED_TO {match_key: row.match_key,
                                            match_level: row.match_level,
                                            match_level_code: row.match_level_code,
                                            is_ambiguous: row.is_ambiguous,
                                            is_disclosed: row.is_disclosed}]->(source)
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_dol_related_rel(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               MATCH (source:Entity {uid: row.source_uid})
               MATCH (target:DOLEntity {uid:row.target_uid})
               MERGE (target)-[:RELATED_TO {match_key: row.match_key,
                                            match_level: row.match_level,
                                            match_level_code: row.match_level_code,
                                            is_ambiguous: row.is_ambiguous,
                                            is_disclosed: row.is_disclosed}]->(source)
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)    

def add_ppp_related_rel(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               MATCH (source:Entity {uid: row.source_uid})
               MATCH (target:PPPEntity {uid:row.target_uid})
               MERGE (target)-[:RELATED_TO {match_key: row.match_key,
                                            match_level: row.match_level,
                                            match_level_code: row.match_level_code,
                                            is_ambiguous: row.is_ambiguous,
                                            is_disclosed: row.is_disclosed}]->(source)
               RETURN count(*) AS total
    """

    return insert_data(query, rows, batch_size)

In [ ]:
add_ent(ent_df_dedup)

In [ ]:
add_ent_rel(resolved_rel_df, batch_size=50)

In [ ]:
sg_related_rel_df = related_rel_df[related_rel_df['target_data_source']=="SAFEGRAPH"]
dol_related_rel_df = related_rel_df[related_rel_df['target_data_source']=="DOL_WHISARD"]
ppp_related_rel_df = related_rel_df[related_rel_df['target_data_source']=="PPP_LOANS"]
sg_related_rel_df.shape, dol_related_rel_df.shape, ppp_related_rel_df.shape

In [ ]:
add_sg_related_rel(sg_related_rel_df)

In [ ]:
add_dol_related_rel(dol_related_rel_df)

In [ ]:
add_ppp_related_rel(ppp_related_rel_df)